In [1]:
import numpy as np
import glob, os
from keras import layers as ly
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import keras.backend.tensorflow_backend as K

Using TensorFlow backend.


In [2]:
dir = './dfs_sequence/*'

In [25]:
# file read
all_data = [] (ㅣ낸 
sequence_length = []
name = []
for file in sorted(glob.glob(dir)):
    name.append(file.split('/')[2].replace('.txt', ''))
    datasets = []
    for f in open(file, 'r'):
        (u, v, w) = f[1:-2].split(',')
        datasets.append([int(u), int(v), float(w)])
    sequence_length.append(len(datasets))
    all_data.append(datasets)
#all_data = np.array(all_data)
all_data = np.array([np.array(arr) for arr in all_data])

In [31]:
x_train, x_test, train_name, test_name = train_test_split(all_data, name, test_size=0.3)
x_test, x_val, test_name, val_name = train_test_split(x_test, test_name, test_size=0.33)

In [34]:
n_features = 3
#batch_size = 32
epochs = 500
steps_per_epoch = len(x_train)

In [46]:
from keras.optimizers import Adam
from keras.layers import Input, LSTM, RepeatVector, Lambda
from keras.models import Model

def repeat_vector(args):
    layer_to_repeat = args[0]
    sequence_layer = args[1]
    return RepeatVector(K.shape(sequence_layer)[1])(layer_to_repeat)
    

inputs = Input(shape=(None, n_features))
encoded = LSTM(128, return_sequences=True)(inputs)  #activation 안적으면 tanh
encoded = LSTM(64, return_sequences=True)(encoded)
encoded = LSTM(32, activation='relu')(encoded)

decoded = Lambda(repeat_vector, output_shape=(None, 32)) ([encoded, inputs]) # inputs의 shape[1] 만큼 encoded 를 반복 생성

decoded = LSTM(32, return_sequences=True)(decoded)
decoded = LSTM(64, return_sequences=True)(decoded)
decoded = LSTM(128, return_sequences=True)(decoded)
decoded = TimeDistributed(Dense(n_features))(decoded)
encoder = Model(inputs, encoded)

lstm_autoencoder = Model(inputs, decoded)
lstm_autoencoder.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001))

def train_generator(x_train):
    idx = 0
    while True:
        yield np.array([x_train[idx]]), np.array([x_train[idx]])
        idx +=1
        if idx >= len(x_train):
            idx = 0
lstm_autoencoder.fit_generator(train_generator(x_train), epochs=epochs, steps_per_epoch=steps_per_epoch, verbose=1)#, validation_data=val_generator(x_val))

''' result
    epoch 500, 0.0624
    epoch 1000, 0.0448
    (256, 128, 64, 64, 128, 256) epoch 100 5.2650 
    (128, 64, 32, 32, 64, 128) epoch 100 2.6025 / 500 0.1083 # activation = tanh
    (128, 64, 32, 32, 64, 128) epoch 100 4.7124 / 500 ? # activation = relu
    
    new data set
    (128, 64, 32, 32, 64, 128) epoch 100 1.9166 / 500  0.0135
'''

Epoch 1/500
661/661 [==============================] - 15s 23ms/step - loss: 47.3037
Epoch 2/500
661/661 [==============================] - 10s 15ms/step - loss: 32.8569
Epoch 3/500
661/661 [==============================] - 10s 15ms/step - loss: 29.9195
Epoch 4/500
661/661 [==============================] - 10s 15ms/step - loss: 26.4544
Epoch 5/500
661/661 [==============================] - 10s 15ms/step - loss: 24.0121
Epoch 6/500
661/661 [==============================] - 10s 15ms/step - loss: 22.6473
Epoch 7/500
661/661 [==============================] - 10s 16ms/step - loss: 21.4523
Epoch 8/500
661/661 [==============================] - 10s 15ms/step - loss: 19.9396
Epoch 9/500
661/661 [==============================] - 10s 16ms/step - loss: 18.8446
Epoch 10/500
661/661 [==============================] - 11s 16ms/step - loss: 18.6257
Epoch 11/500
661/661 [==============================] - 11s 16ms/step - loss: 19.7394
Epoch 12/500
661/661 [==============================] - 10s 16m

661/661 [==============================] - 10s 15ms/step - loss: 2.8607
Epoch 98/500
661/661 [==============================] - 10s 15ms/step - loss: 2.5616
Epoch 99/500
661/661 [==============================] - 10s 15ms/step - loss: 2.5750
Epoch 100/500
661/661 [==============================] - 10s 16ms/step - loss: 4.7124
Epoch 101/500
661/661 [==============================] - 10s 16ms/step - loss: 4.0930
Epoch 102/500
661/661 [==============================] - 10s 15ms/step - loss: 2.7874
Epoch 103/500
661/661 [==============================] - 10s 15ms/step - loss: 2.5025
Epoch 104/500
661/661 [==============================] - 10s 15ms/step - loss: 2.1881
Epoch 105/500
661/661 [==============================] - 10s 15ms/step - loss: 3.3412
Epoch 106/500
661/661 [==============================] - 10s 15ms/step - loss: 4.1738
Epoch 107/500
661/661 [==============================] - 10s 15ms/step - loss: 3.0216
Epoch 108/500
661/661 [==============================] - 10s 15ms/step

661/661 [==============================] - 11s 16ms/step - loss: 0.9258
Epoch 193/500
661/661 [==============================] - 11s 16ms/step - loss: 1.1437
Epoch 194/500
661/661 [==============================] - 11s 16ms/step - loss: 0.8914
Epoch 195/500
661/661 [==============================] - 11s 16ms/step - loss: 0.6899
Epoch 196/500
661/661 [==============================] - 11s 16ms/step - loss: 0.7247
Epoch 197/500
661/661 [==============================] - 11s 16ms/step - loss: 1.0458
Epoch 198/500
661/661 [==============================] - 11s 16ms/step - loss: 1.9995
Epoch 199/500
661/661 [==============================] - 13s 20ms/step - loss: 1.5130
Epoch 200/500
661/661 [==============================] - 12s 18ms/step - loss: 0.7401
Epoch 201/500
661/661 [==============================] - 10s 16ms/step - loss: 0.7354
Epoch 202/500
661/661 [==============================] - 10s 16ms/step - loss: 0.8473
Epoch 203/500
661/661 [==============================] - 10s 15ms/st

661/661 [==============================] - 8s 12ms/step - loss: 0.2380
Epoch 289/500
661/661 [==============================] - 8s 11ms/step - loss: 0.1939
Epoch 290/500
661/661 [==============================] - 8s 12ms/step - loss: 0.1731
Epoch 291/500
661/661 [==============================] - 8s 11ms/step - loss: 0.2065
Epoch 292/500
661/661 [==============================] - 7s 11ms/step - loss: 0.8207
Epoch 293/500
661/661 [==============================] - 7s 11ms/step - loss: 0.9591
Epoch 294/500
661/661 [==============================] - 7s 11ms/step - loss: 0.4645
Epoch 295/500
661/661 [==============================] - 7s 11ms/step - loss: 0.2426
Epoch 296/500
661/661 [==============================] - 7s 11ms/step - loss: 0.1869
Epoch 297/500
661/661 [==============================] - 7s 11ms/step - loss: 0.2004
Epoch 298/500
661/661 [==============================] - 7s 11ms/step - loss: 0.3480
Epoch 299/500
661/661 [==============================] - 7s 11ms/step - loss: 0

661/661 [==============================] - 10s 15ms/step - loss: 0.2087
Epoch 385/500
661/661 [==============================] - 10s 15ms/step - loss: 0.2813
Epoch 386/500
661/661 [==============================] - 10s 15ms/step - loss: 0.2933
Epoch 387/500
661/661 [==============================] - 10s 15ms/step - loss: 0.2652
Epoch 388/500
661/661 [==============================] - 10s 15ms/step - loss: 0.2367
Epoch 389/500
661/661 [==============================] - 10s 15ms/step - loss: 0.2678
Epoch 390/500
661/661 [==============================] - 10s 15ms/step - loss: 1.8253
Epoch 391/500
661/661 [==============================] - 10s 15ms/step - loss: 1.0596
Epoch 392/500
661/661 [==============================] - 10s 15ms/step - loss: 0.4532
Epoch 393/500
661/661 [==============================] - 10s 15ms/step - loss: 0.2349
Epoch 394/500
661/661 [==============================] - 10s 15ms/step - loss: 0.1266
Epoch 395/500
661/661 [==============================] - 10s 15ms/st

661/661 [==============================] - 10s 16ms/step - loss: 0.2662
Epoch 480/500
661/661 [==============================] - 10s 16ms/step - loss: 1.6743
Epoch 481/500
661/661 [==============================] - 11s 16ms/step - loss: 1.3653
Epoch 482/500
661/661 [==============================] - 11s 16ms/step - loss: 0.5010
Epoch 483/500
661/661 [==============================] - 10s 15ms/step - loss: 0.2269
Epoch 484/500
661/661 [==============================] - 10s 15ms/step - loss: 0.1584
Epoch 485/500
661/661 [==============================] - 10s 15ms/step - loss: 0.1182
Epoch 486/500
661/661 [==============================] - 10s 15ms/step - loss: 0.0902
Epoch 487/500
661/661 [==============================] - 10s 15ms/step - loss: 0.0793
Epoch 488/500
661/661 [==============================] - 10s 15ms/step - loss: 0.0832
Epoch 489/500
661/661 [==============================] - 10s 15ms/step - loss: 0.0960
Epoch 490/500
661/661 [==============================] - 10s 15ms/st

' result\n    epoch 500, 0.0624\n    epoch 1000, 0.0448\n    (256, 128, 64, 64, 128, 256) epoch 100 5.2650 \n    (128, 64, 32, 32, 64, 128) epoch 100 2.6025 / 500 0.1083 # activation = tanh\n    (128, 64, 32, 32, 64, 128) epoch 100 ? / 500 ? # activation = relu\n    \n    new data set\n    (128, 64, 32, 32, 64, 128) epoch 100 1.9166 / 500 \n'

In [48]:
from keras.models import load_model
# model save
lstm_autoencoder.save('lstm_autoencoder_prev_dfs.h5')
loaded_model = load_model('lstm_autoencoder_prev_dfs.h5')

In [ ]:
encoder = Model(inputs, encoded)

In [50]:
loaded_model.summary()

Model: "model_16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, None, 3)      0                                            
__________________________________________________________________________________________________
lstm_35 (LSTM)                  (None, None, 128)    67584       input_8[0][0]                    
__________________________________________________________________________________________________
lstm_36 (LSTM)                  (None, None, 64)     49408       lstm_35[0][0]                    
__________________________________________________________________________________________________
lstm_37 (LSTM)                  (None, 32)           12416       lstm_36[0][0]                    
___________________________________________________________________________________________

In [45]:
lstm_autoencoder.summary()
#lstm_autoencoder.layers[3].activation

Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, None, 3)      0                                            
__________________________________________________________________________________________________
lstm_29 (LSTM)                  (None, None, 128)    67584       input_7[0][0]                    
__________________________________________________________________________________________________
lstm_30 (LSTM)                  (None, None, 64)     49408       lstm_29[0][0]                    
__________________________________________________________________________________________________
lstm_31 (LSTM)                  (None, 32)           12416       lstm_30[0][0]                    
___________________________________________________________________________________________

<function keras.activations.relu(x, alpha=0.0, max_value=None, threshold=0.0)>

In [17]:
encoder.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, None, 3)           0         
_________________________________________________________________
lstm_9 (LSTM)                (None, None, 128)         67584     
_________________________________________________________________
lstm_10 (LSTM)               (None, 64)                49408     
Total params: 116,992
Trainable params: 116,992
Non-trainable params: 0
_________________________________________________________________


In [19]:
x_test[0]

array([[ 2.  ,  1.  ,  7.84],
       [ 1.  ,  3.  ,  6.79],
       [ 3.  ,  2.  , 22.  ],
       [ 1.  ,  4.  , 11.79],
       [ 2.  ,  5.  , 11.09]])

In [42]:
lstm_autoencoder.predict(x_test[0].reshape(1, x_test[0].shape[0], x_test[0].shape[1]))

array([[[ 8.931337 ,  7.4579573, 31.943764 ],
        [ 8.431695 ,  7.737891 ,  8.375352 ],
        [ 7.3948007,  5.0614414, 13.785835 ],
        [ 7.925072 ,  5.933628 , 14.485416 ],
        [ 2.0842404,  1.931783 ,  8.705457 ],
        [ 2.2869372,  3.9760273,  7.5900183],
        [ 7.5529623,  5.3252344, 26.848215 ],
        [ 2.7739074,  3.8996158,  8.020057 ],
        [ 1.5346651,  4.882212 , 11.469937 ]]], dtype=float32)

In [76]:
test = x_test[0].reshape(1, 9, 3)
latent_vector = []
for x in x_test:
    x = x.reshape(1, x.shape[0], x.shape[1])
    latent_vector.append(encoder.predict(x)[0])

In [144]:
def printMatrix(matrix):
    length = int(max(max(matrix[:, 0]), max(matrix[:, 1])))
    adj = [[0 for i in range(length)] for i in range(length)]
    
    for m in matrix:
        (i, j, w) = m
        i = int(i) -1
        j = int(j) -1
        adj[i][j] = adj[j][i] = w
    for i in range(length):
        for j in range(length):
            print(str(adj[i][j]), end=',')
        print()
    print("===============")

In [213]:
# sequence to matrix
for index, x in enumerate(x_test):
    print("#", str(index))
    printMatrix(x)
    

# 0
0,10.16,7.92,9.56,0,0,0,0,0,
10.16,0,0,0,0,0,0,0,0,
7.92,0,0,0,8.17,18.05,0,0,0,
9.56,0,0,0,0,0,20.05,0,0,
0,0,8.17,0,0,0,0,0,0,
0,0,18.05,0,0,0,0,32.09,0,
0,0,0,20.05,0,0,0,0,20.27,
0,0,0,0,0,32.09,0,0,9.01,
0,0,0,0,0,0,20.27,9.01,0,
# 1
0,8.79,11.79,0,0,0,
8.79,0,0,10.06,6.86,0,
11.79,0,0,0,0,7.86,
0,10.06,0,0,0,0,
0,6.86,0,0,0,0,
0,0,7.86,0,0,0,
# 2
0,3.3,6.3,0,0,
3.3,0,0,12.72,0,
6.3,0,0,0,23.87,
0,12.72,0,0,41.95,
0,0,23.87,41.95,0,
# 3
0,5.7,7.22,0,0,
5.7,0,0,12.22,0,
7.22,0,0,0,23.54,
0,12.22,0,0,42.39,
0,0,23.54,42.39,0,
# 4
0,12.52,17.52,28.77,0,0,0,0,0,
12.52,0,0,0,0,0,0,0,0,
17.52,0,0,0,2.53,4.53,0,0,0,
28.77,0,0,0,0,0,10.86,0,0,
0,0,2.53,0,0,0,0,0,0,
0,0,4.53,0,0,0,0,9.01,0,
0,0,0,10.86,0,0,0,0,10.01,
0,0,0,0,0,9.01,0,0,20.01,
0,0,0,0,0,0,10.01,20.01,0,
# 5
0,4.27,9.29,0,0,
4.27,0,0,12.29,0,
9.29,0,0,0,17.29,
0,12.29,0,0,3.95,
0,0,17.29,3.95,0,
# 6
0,10.16,7.92,9.56,0,0,0,0,0,
10.16,0,0,0,0,0,0,0,0,
7.92,0,0,0,8.17,18.05,0,0,0,
9.56,0,0,0,0,0,20.05,0,0,
0,0,8.17,0,0,0,0

In [171]:
standard = latent_vector[0]

In [223]:
import copy
dist = []
left = 100000
right = -10000
for index, v in enumerate(latent_vector[1:]):
    d = round(euclidean_distance(standard, v), 8)
    dist.append(d)
    if d > right:
        right= d
    if left > d:
        left = d
dist1 = copy.deepcopy(dist)
dist1.sort()
#print(dist, dist1)
for index, d in enumerate(dist1):
    print( (index+1, dist.index(d)+1), end='  ')

(1, 2)  (2, 38)  (3, 53)  (4, 5)  (5, 15)  (6, 44)  (7, 48)  (8, 21)  (9, 12)  (10, 43)  (11, 9)  (12, 18)  (13, 31)  (14, 59)  (15, 58)  (16, 4)  (17, 41)  (18, 27)  (19, 29)  (20, 33)  (21, 6)  (22, 3)  (23, 51)  (24, 35)  (25, 54)  (26, 55)  (27, 26)  (28, 62)  (29, 14)  (30, 46)  (31, 11)  (32, 45)  (33, 7)  (34, 50)  (35, 57)  (36, 8)  (37, 63)  (38, 60)  (39, 24)  (40, 52)  (41, 20)  (42, 13)  (43, 37)  (44, 23)  (45, 10)  (46, 1)  (47, 61)  (48, 42)  (49, 56)  (50, 34)  (51, 16)  (52, 32)  (53, 40)  (54, 39)  (55, 19)  (56, 17)  (57, 22)  (58, 49)  (59, 30)  (60, 36)  (61, 47)  (62, 28)  (63, 25)  

In [168]:
def euclidean_distance(a, b):
    return np.sqrt(np.sum(a-b) ** 2)

In [221]:
len(x_test)

64